# Tutorial 2: Broadcasting and Performance Optimization

In this tutorial, we demonstrate how `lsbi` can be used to efficiently analyze thousands of datasets without explicit `for` loops, and how to gain significant speedups using diagonal covariance optimizations.

We will cover:
1. The scenario: analyzing many experiments simultaneously
2. The slow way: using a `for` loop
3. The fast way: `lsbi` broadcasting
4. Performance optimization with diagonal covariances
5. Best practices for high-dimensional problems

## 1. The Scenario: Many Experiments

Imagine you are running the same line-fitting experiment from Tutorial 1, but you run it `5000` times, each time getting a new noisy dataset. Your goal is to find the posterior for each of these `5000` experiments.

In [ ]:
# Setup for many experiments
import numpy as np
from lsbi import LinearModel
import time

# Use the same model setup as Tutorial 1
np.random.seed(0)
theta_true = np.array([2.5, -1.0])
n_params = len(theta_true)
x_data = np.linspace(-2, 2, 20)
n_data = len(x_data)

M = np.vstack([x_data, np.ones_like(x_data)]).T
mu = np.zeros(n_params)
Sigma = np.eye(n_params) * 10
data_noise_std = 0.5
C = np.eye(n_data) * data_noise_std**2

model = LinearModel(M=M, mu=mu, Sigma=Sigma, C=C)

# Generate 5000 noisy datasets
num_datasets = 5000
y_data_true = M @ theta_true
all_y_data_noisy = y_data_true + np.random.normal(0, data_noise_std, size=(num_datasets, n_data))
print("Shape of all datasets:", all_y_data_noisy.shape)
print(f"Total data points: {num_datasets * n_data:,}")

## 2. The Slow Way: Using a `for` loop

The naive approach is to loop over each dataset and compute the posterior individually. Let's see how long this takes:

In [ ]:
# The slow for-loop approach
print("Computing posteriors using a for-loop...")
start_time = time.time()
posterior_means_loop = []
for i in range(num_datasets):
    D = all_y_data_noisy[i]
    posterior = model.posterior(D)
    posterior_means_loop.append(posterior.mean)
    
    # Progress indicator
    if (i + 1) % 1000 == 0 or i == 0:
        print(f"  Processed {i + 1:4d}/{num_datasets} datasets")
        
end_time = time.time()
loop_time = end_time - start_time

print(f"\nFor-loop approach took: {loop_time:.4f} seconds")
print(f"Average time per dataset: {loop_time/num_datasets*1000:.2f} ms")

# Shape of result is a list of arrays, we can stack them for comparison
posterior_means_loop = np.array(posterior_means_loop)
print("Shape of stacked results:", posterior_means_loop.shape)

## 3. The Fast Way: `lsbi` Broadcasting

`lsbi` is vectorized. We can pass the entire batch of datasets `(5000, 20)` directly to the `posterior` method. `lsbi` will automatically broadcast the calculation over the first dimension.

In [ ]:
# The fast broadcasting approach
print("Computing posteriors using broadcasting...")
start_time = time.time()
# Pass the entire batch of data at once
batch_posterior = model.posterior(all_y_data_noisy)
end_time = time.time()
broadcast_time = end_time - start_time

print(f"Broadcasting approach took: {broadcast_time:.4f} seconds")
print(f"Average time per dataset: {broadcast_time/num_datasets*1000:.2f} ms")

# Check the output shape
print("\nShape of batch_posterior:", batch_posterior.shape)
print("Shape of batch_posterior.mean:", batch_posterior.mean.shape)

# Verify the results are identical
np.testing.assert_allclose(posterior_means_loop, batch_posterior.mean, rtol=1e-10)
print("\n✓ Results are numerically identical!")

# Show the speedup
speedup = loop_time / broadcast_time
print(f"\n🚀 Speedup: {speedup:.1f}x faster with broadcasting!")

The broadcasting approach is not only cleaner but dramatically more performant! This speedup comes from:
- **Vectorized operations**: NumPy's C-optimized routines
- **Reduced Python overhead**: Single function call instead of 5000
- **Better memory access patterns**: More cache-friendly operations

## 4. Performance with Diagonal Covariances

`lsbi` includes special, faster routines for when the covariance matrices `Σ` and `C` are diagonal. This is common when parameters or data points are assumed to be uncorrelated.

Let's create a higher-dimensional problem to see the effect.

In [ ]:
# Create a high-dimensional problem
print("Setting up high-dimensional problem...")
np.random.seed(1)
n_params_hd = 100
n_data_hd = 500
n_experiments = 100  # Smaller number for timing

# Create a high-dimensional problem with diagonal covariances
M_hd = np.random.rand(n_data_hd, n_params_hd)
mu_hd = np.zeros(n_params_hd)
Sigma_hd_diag = np.ones(n_params_hd)  # Prior variance is 1 for all params
C_hd_diag = np.ones(n_data_hd) * 0.1   # Data variance is 0.1 for all points

# Generate test data
D_hd_batch = np.random.rand(n_experiments, n_data_hd)

print(f"Problem size: {n_params_hd} parameters, {n_data_hd} data points, {n_experiments} experiments")
print(f"Total matrix operations: {n_experiments} × {n_data_hd}×{n_params_hd} = {n_experiments*n_data_hd*n_params_hd:,} elements")

In [ ]:
# Case 1: Using full covariance matrices
print("\n=== Full Matrix Version ===")
model_full = LinearModel(M=M_hd, mu=mu_hd, 
                        Sigma=np.diag(Sigma_hd_diag), 
                        C=np.diag(C_hd_diag))

print("Computing posteriors with full matrices...")
t_start_full = time.time()
post_full = model_full.posterior(D_hd_batch)
t_end_full = time.time()
full_time = t_end_full - t_start_full
print(f"Full matrix version took: {full_time:.4f} seconds")

print(f"Memory usage - Sigma: {np.diag(Sigma_hd_diag).nbytes / 1024**2:.1f} MB")
print(f"Memory usage - C: {np.diag(C_hd_diag).nbytes / 1024**2:.1f} MB")

In [ ]:
# Case 2: Using the diagonal optimization
print("\n=== Diagonal Optimization ===")
model_diag = LinearModel(M=M_hd, mu=mu_hd, 
                        Sigma=Sigma_hd_diag, C=C_hd_diag,
                        diagonal_Sigma=True, diagonal_C=True)

print("Computing posteriors with diagonal optimization...")
t_start_diag = time.time()
post_diag = model_diag.posterior(D_hd_batch)
t_end_diag = time.time()
diag_time = t_end_diag - t_start_diag
print(f"Diagonal optimization took: {diag_time:.4f} seconds")

print(f"Memory usage - Sigma: {Sigma_hd_diag.nbytes / 1024:.1f} KB")
print(f"Memory usage - C: {C_hd_diag.nbytes / 1024:.1f} KB")

# Verify results are identical
np.testing.assert_allclose(post_full.mean, post_diag.mean, rtol=1e-10)
np.testing.assert_allclose(post_full.cov, post_diag.cov, rtol=1e-10)
print("\n✓ Results for full and diagonal versions are numerically identical!")

# Show the performance and memory improvements
speedup_diag = full_time / diag_time
memory_reduction = (np.diag(Sigma_hd_diag).nbytes + np.diag(C_hd_diag).nbytes) / (Sigma_hd_diag.nbytes + C_hd_diag.nbytes)

print(f"\n🚀 Diagonal optimization speedup: {speedup_diag:.1f}x")
print(f"💾 Memory reduction: {memory_reduction:.1f}x less memory used")

**Common Pitfall:** When setting `diagonal_C=True` or `diagonal_Sigma=True`, you **must** pass 1D arrays for `C` and `Sigma` containing only the diagonal elements. Passing a full 2D matrix will cause an error.

**Correct:**
```python
# For diagonal matrix with values [1, 2, 3] on diagonal
LinearModel(C=np.array([1, 2, 3]), diagonal_C=True)  # ✓ Correct
```

**Incorrect:**
```python
LinearModel(C=np.diag([1, 2, 3]), diagonal_C=True)  # ✗ Wrong!
```

## 5. Visualizing the Performance Scaling

Let's see how the performance benefits scale with problem size:

In [ ]:
# Performance scaling experiment
import matplotlib.pyplot as plt

print("Running performance scaling experiment...")
dimensions = [10, 25, 50, 100, 200]
times_full = []
times_diag = []

for dim in dimensions:
    print(f"  Testing dimension {dim}...")
    
    # Setup
    M_test = np.random.rand(dim, dim)
    D_test = np.random.rand(10, dim)  # 10 experiments
    
    # Full matrices
    model_test_full = LinearModel(M=M_test, 
                                 Sigma=np.eye(dim), 
                                 C=np.eye(dim) * 0.1)
    
    start = time.time()
    _ = model_test_full.posterior(D_test)
    times_full.append(time.time() - start)
    
    # Diagonal optimization
    model_test_diag = LinearModel(M=M_test, 
                                 Sigma=np.ones(dim), 
                                 C=np.ones(dim) * 0.1,
                                 diagonal_Sigma=True, 
                                 diagonal_C=True)
    
    start = time.time()
    _ = model_test_diag.posterior(D_test)
    times_diag.append(time.time() - start)

# Plot results
plt.figure(figsize=(10, 6))
plt.loglog(dimensions, times_full, 'b-o', label='Full Matrices', linewidth=2, markersize=8)
plt.loglog(dimensions, times_diag, 'r-s', label='Diagonal Optimization', linewidth=2, markersize=8)
plt.xlabel('Problem Dimension')
plt.ylabel('Time (seconds)')
plt.title('Performance Scaling: Full vs Diagonal Matrices')
plt.legend()
plt.grid(True, alpha=0.3)

# Add speedup annotations
for i, dim in enumerate(dimensions):
    speedup = times_full[i] / times_diag[i]
    plt.annotate(f'{speedup:.1f}x', 
                xy=(dim, times_diag[i]), 
                xytext=(dim, times_diag[i] * 0.5),
                ha='center', fontsize=10,
                arrowprops=dict(arrowstyle='->', color='green', lw=1))

plt.tight_layout()
plt.show()

print(f"\nMaximum speedup observed: {max(np.array(times_full)/np.array(times_diag)):.1f}x")

## 6. Best Practices for High-Performance `lsbi`

Based on what we've learned, here are the key performance optimization strategies:

In [ ]:
# Performance best practices demonstration
print("=== Performance Best Practices ===")
print()
print("1. 📊 USE BROADCASTING:")
print("   ✓ Pass batched data: shape (N, d) for N datasets")
print("   ✗ Avoid: for loops over individual datasets")
print()
print("2. 🔢 USE DIAGONAL OPTIMIZATIONS:")
print("   ✓ When parameters are independent: diagonal_Sigma=True")
print("   ✓ When data points are independent: diagonal_C=True")
print("   ✓ When model is diagonal: diagonal_M=True")
print()
print("3. 💾 MEMORY EFFICIENCY:")
print("   ✓ Use 1D arrays for diagonal matrices")
print("   ✓ Consider float32 for large problems if precision allows")
print()
print("4. 🎯 WHEN TO USE EACH APPROACH:")
print("   • Small problems (< 100 params): Either approach is fine")
print("   • Medium problems (100-1000 params): Diagonal if applicable")
print("   • Large problems (> 1000 params): Diagonal essential")
print("   • Many datasets: Always use broadcasting")

# Quick benchmark summary
print("\n=== Benchmark Summary ===")
print(f"Broadcasting speedup: {speedup:.1f}x (5000 datasets)")
print(f"Diagonal speedup: {max(np.array(times_full)/np.array(times_diag)):.1f}x (200D problem)")
print(f"Combined potential: {speedup * max(np.array(times_full)/np.array(times_diag)):.0f}x+ speedup possible!")

## Summary

In this tutorial, we explored `lsbi`'s powerful performance features:

### Key Takeaways:

1. **Broadcasting is Essential**: Pass all your datasets at once rather than looping. This can provide 50-100x+ speedups with no change in results.

2. **Diagonal Optimizations Matter**: When your covariance matrices are diagonal (uncorrelated parameters/data), use the `diagonal_*=True` flags for significant speedups and memory savings.

3. **Scale Matters**: Performance benefits become more pronounced with larger problems. For high-dimensional problems, diagonal optimizations can mean the difference between feasible and infeasible computations.

4. **Memory Efficiency**: Diagonal optimizations reduce memory usage by factors of 100+ for large problems, enabling analysis that wouldn't fit in memory otherwise.

### When to Use What:

- **Always use broadcasting** when analyzing multiple datasets
- **Use diagonal optimizations** when your problem structure allows it
- **Combine both** for maximum performance on large-scale problems

Next, try Tutorial 3 to learn about handling complex, multi-modal problems with `MixtureModel`!